# OpenAI Function Calling in LangChain

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic: Data validation library for Python

In [ ]:
# Normal class
class User:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

foo = User(name="Joe", age="x") # works
print("Object:", foo)
print("Name:", foo.name)
print("Age:", foo.age)

In [ ]:
class pUser(BaseModel):
    name: str
    age: int

# foo = pUser(name="Joe", age="x") # error
foo = pUser(name="Joe", age=32) # works
foo # pretty print

In [ ]:
# List of objects
class pClass(BaseModel):
    students: List[pUser]

obj = pClass(
    students=[
        foo,
        pUser(name="Jane", age=35),
    ]
)
obj

## Use Pydantic to define OpenAI fucntions

In [ ]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at the airport""" # function description, required
    airport_code: str = Field(description="airport code to get the weater for") # function params, the desc is not required

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

weather_function = convert_to_openai_function(WeatherSearch)
weather_function

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI().bind(functions=[weather_function], function_call={"name": "WeatherSearch"})
print(model.invoke("What is the weather in CDG today?"))
print(model.invoke("Hi"))

## Pass multiple functions

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

artist_search = convert_to_openai_function(ArtistSearch)

In [ ]:
functions = [weather_function, artist_search]
model = ChatOpenAI().bind(functions=functions)

In [ ]:
print(model.invoke("What's the weather in Orly?"))
print(model.invoke("What are five songs by Dinos?"))
print(model.invoke("Hi!"))